In [ ]:
#Impleneting a hands-free mouse control system using computer vision and facial
#landmark detection by using OpenCV, MediaPipe FaceMesh, and PyAutoGUI. The system enables users to move the mouse
#cursor using eye movement and perform clicks through blinking. Please use your left eye to click.

In [24]:
# Import necessary libraries
import cv2  # OpenCV for capturing and processing video
import mediapipe as mp  # MediaPipe for facial landmark detection
import pyautogui  # PyAutoGUI for controlling the mouse

In [26]:
# Initialize webcam
cam = cv2.VideoCapture(0)

# Initialize MediaPipe FaceMesh model with refined landmarks for more accuracy
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# Get the screen width and height for mapping eye movement to the screen
screen_w, screen_h = pyautogui.size()


In [ ]:
# Infinite loop to continuously process video frames
while True:
    # Read a frame from the webcam
    _, frame = cam.read()
    
    # Flip the frame horizontally to act as a mirror
    frame = cv2.flip(frame, 1)
    
    # Convert frame from BGR (OpenCV default) to RGB (MediaPipe requires RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame using the FaceMesh model to detect facial landmarks
    output = face_mesh.process(rgb_frame)
    
    # Get detected face landmarks
    landmark_points = output.multi_face_landmarks
    
    # Get frame dimensions
    frame_h, frame_w, _ = frame.shape

    # If facial landmarks are detected
    if landmark_points:
        # Extract the first detected face's landmarks
        landmarks = landmark_points[0].landmark
        
        # Iterate through specific landmarks (474-478), which are around the eyes
        for id, landmark in enumerate(landmarks[474:478]):
            # Convert normalized landmark coordinates to pixel values
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            
            # Draw a small green circle around the detected eye landmarks
            cv2.circle(frame, (x, y), 3, (0, 255, 0))
            
            # Use one specific landmark (id == 1) to control mouse movement
            if id == 1:
                # Map facial landmark coordinates to screen coordinates
                screen_x = screen_w * landmark.x
                screen_y = screen_h * landmark.y
                
                # Move the mouse cursor to the calculated screen coordinates
                pyautogui.moveTo(screen_x, screen_y)

        # Select two specific landmarks for eye blinking detection (145 and 159)
        left = [landmarks[145], landmarks[159]]

        # Draw circles on these landmarks for visualization
        for landmark in left:
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 255))  # Yellow color

        # Detect blink based on vertical eye distance threshold
        if (left[0].y - left[1].y) < 0.004:
            # Perform a mouse click if blink is detected
            pyautogui.click()
            # Add a small delay to prevent multiple clicks from a single blink
            pyautogui.sleep(1)

    # Display the video feed with annotations
    cv2.imshow('Eye Controlled Mouse', frame)
    
    # Wait for a key press for 1ms (non-blocking)
    cv2.waitKey(1)